In [1]:
import pandas as pd

In [2]:
import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('final_data.csv')

In [4]:
data.describe()

,pH,N,P,K,OC,Particles,Water_holding_content
count,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000,999.000000
mean,6.498879,69.971431,10.946186,146.753734,0.503173,0.269286,1.722392
std,2.178742,28.300420,7.517791,35.535056,0.266441,0.511614,0.462599
min,0.030000,25.060000,0.610000,100.040000,0.010000,0.000017,1.200000
25%,5.755000,48.895000,4.375000,109.640000,0.290000,0.001569,1.320000
50%,7.630000,65.510000,9.270000,144.330000,0.510000,0.023609,1.430000
75%,7.820000,88.405000,17.760000,178.750000,0.730000,0.115000,2.210000
max,8.000000,134.860000,25.000000,209.810000,0.950000,1.990000,2.500000


In [5]:
data.head()

,pH,N,P,K,OC,Particles,Water_holding_content,Soil_type,crop_type
0,7.03,118.72,25.00,185.82,0.78,0.001686,1.40,clay,carrot
1,2.57,131.09,24.62,181.71,0.90,0.000381,1.31,clay,carrot
2,3.39,104.32,18.37,202.39,0.76,0.000226,1.35,clay,carrot
3,2.73,111.21,21.09,202.71,0.68,0.001399,1.22,clay,carrot
4,1.52,90.90,23.76,195.39,0.74,0.001587,1.21,clay,carrot
